# One-Step Lookahead
Make your agent smarter with a few simple changes.

# 🎯 Introduction  
In this project, you will learn how to teach a Connect Four agent to make smart moves by using a **heuristic** — a simple way to estimate which boards are good or bad.

# 🌳 Game Trees
How do humans decide their moves in Connect Four? 🤔
- 🔮 They forecast future moves: "If I play here, the opponent will likely do that, and then I could respond like this..."
- 🧠 This thinking process can be represented with a **game tree**, showing all possible moves from an empty board.

> **Problem**:  
> The complete game tree is **huge** (over **4 trillion** different boards!).  
> 🚫 It's impossible to explore the full tree during the game.

# 🛠️ Heuristics to the Rescue!
Since exploring everything is impossible, we need a shortcut: a **heuristic**.
- 📈 A heuristic assigns a **score** to any board.
- ⚡ Higher scores mean better chances for the agent to win.

# 🧩 Example: A Simple Heuristic
Scan all rows, columns, and diagonals, and apply:
- 🏆 **+1,000,000 points**: if the agent has **4 discs in a row** (winning!).
- 🎯 **+1 point**: if the agent has **3 discs and 1 empty spot** (almost winning).
- ⚠️ **-100 points**: if the opponent has **3 discs and 1 empty spot** (danger!).

# 🤖 How the Agent Makes a Move
At each turn:
1. 🕹️ Simulate all 7 possible moves (one per column).
2. 📊 Calculate the heuristic score for the resulting board.
3. 🏅 Choose the move with the **highest score**!

### 🧪 Example Walkthrough
Imagine it's the agent’s turn:
- Move in column 0 → score = **2** (two favorable patterns found).
- Move in column 1 → score = **1**.
- Move in column 2 → score = **0**.

✅ The agent picks **column 0** — the best move!

# 💡 Tips for Designing Your Own Heuristic
- 🎨 **Start simple**: create an initial version of your heuristic.
- 🧑‍💻 **Play against your agent** to observe mistakes.
- 🛠️ **Tweak the scoring rules** based on what you notice.
- 🚀 **Iterate and improve** until your agent plays smarter!

---

# 📋 Quick Summary

| Situation                                  | Score         |
|--------------------------------------------|---------------|
| 4 in a row (agent wins)                    | +1,000,000 🏆 |
| 3 in a row (agent) + 1 empty                | +1 🎯         |
| 3 in a row (opponent) + 1 empty             | -100 ⚠️       |

---


### 1) A more complex heuristic

The previous heuristic looks at all groups of four adjacent grid locations on the same row, column, or diagonal and assigns points for each occurrence of the following patterns:
<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/vzQa4ML.png" width=60%><br/>
</center>
In the image above, we assume that the agent is the red player, and the opponent plays yellow discs.

For reference, here is the previous heuristic `get_heuristic()` :
```python
def get_heuristic(grid, mark, config):
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    score = num_threes - 1e2*num_threes_opp + 1e6*num_fours
    return score
```

In the `get_heuristic()` function, `num_fours`, `num_threes`, and `num_threes_opp` are the number of windows in the game grid that are assigned 1000000, 1, and -100 point(s), respectively. 

Now, we'll change the heuristic to the following (where we decide the number of points to apply in each of `A`, `B`, `C`, `D`, and `E`).
<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/FBoWr2f.png" width=80%><br/>
</center>

So our updated heuristic function looks like this:
```python
def get_heuristic_1(grid, col, mark, config):
    num_twos = count_windows(grid, 2, mark, config)
    num_threes = count_windows(grid, 3, mark, config)
    num_fours = count_windows(grid, 4, mark, config)
    num_twos_opp = count_windows(grid, 2, mark%2+1, config)
    num_threes_opp = count_windows(grid, 3, mark%2+1, config)
    score = A*num_fours + B*num_threes + C*num_twos + D*num_twos_opp + E*num_threes_opp
    return score
```

This heuristic is then used to create an agent, that competes against the agent from the tutorial in 50 different game rounds.  In order to be marked correct, 
- our agent must win at least half of the games, and
- `C` and `D` must both be nonzero.

In [1]:
A = 1e10
B = 1e4
C = 1e2
D = -1
E = -1e6

### 2) Does the agent win?

Consider the game board below.  
<center>
<img src="https://storage.googleapis.com/kaggle-media/learn/images/AlnaQ3J.png" width=30%><br/>
</center>

Say the agent uses red discs, and it's the agent's turn.  
- If the agent uses the first heuristic, does it win or lose the game?
- If the agent uses the heuristic **_that we just implemented_**, does it win or lose the game?

**Solution**: The agent has two choices: it can play in either column 0 (the leftmost column), or column 6 (the rightmost column). If the agent plays in column 0, it definitely wins the game in its next move. And, if it plays in column 6, it likely loses the game (since, if the opponent responds by playing in the same column, then the opponent wins the game).

If the agent uses the first heuristic, both columns are scored equally, and so the agent will select from them (uniformly) at random. In this case, the agent has about a 50/50 chance of winning the game.

As for the second heuristic, this will depend on your implementation, so we'll provide an answer for the solution heuristic that we provided -- in this case, the agent most likely loses the game, since it will definitely select the final column.

This is an interesting situation, because on average, we see that the agent with the new heuristic performs better than the agent from the tutorial (and yet, for this board, it's guaranteed to make the wrong decision).

In [2]:
def my_agent(obs, config):
    import random
    import numpy as np


    def count_windows(grid, num_discs, mark, config):
        count = 0
        # Horizontal
        for row in range(config.rows):
            for col in range(config.columns - 3):
                window = [grid[row][col+i] for i in range(4)]
                if window.count(mark) == num_discs and window.count(0) == 4 - num_discs:
                    count += 1
        # Vertical
        for row in range(config.rows - 3):
            for col in range(config.columns):
                window = [grid[row+i][col] for i in range(4)]
                if window.count(mark) == num_discs and window.count(0) == 4 - num_discs:
                    count += 1
        # Positive diagonal
        for row in range(config.rows - 3):
            for col in range(config.columns - 3):
                window = [grid[row+i][col+i] for i in range(4)]
                if window.count(mark) == num_discs and window.count(0) == 4 - num_discs:
                    count += 1
        # Negative diagonal
        for row in range(3, config.rows):
            for col in range(config.columns - 3):
                window = [grid[row-i][col+i] for i in range(4)]
                if window.count(mark) == num_discs and window.count(0) == 4 - num_discs:
                    count += 1
        return count

    def get_heuristic_1(grid, mark, config):
        num_twos = count_windows(grid, 2, mark, config)
        num_threes = count_windows(grid, 3, mark, config)
        num_fours = count_windows(grid, 4, mark, config)
        num_twos_opp = count_windows(grid, 2, 3 - mark, config)
        num_threes_opp = count_windows(grid, 3, 3 - mark, config)
        score = A*num_fours + B*num_threes + C*num_twos + D*num_twos_opp + E*num_threes_opp
        return score

    def drop_piece(grid, row, col, mark):
        grid[row][col] = mark

    def get_next_open_row(grid, col, config):
        for row in range(config.rows):
            if grid[row][col] == 0:
                return row
        return None

    # Prepare board
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)

    # All valid columns
    valid_moves = [col for col in range(config.columns) if grid[config.rows-1][col] == 0]

    # Choose the best move based on heuristic
    best_score = float('-inf')
    best_move = random.choice(valid_moves)
    for col in valid_moves:
        temp_grid = grid.copy()
        row = get_next_open_row(temp_grid, col, config)
        if row is not None:
            drop_piece(temp_grid, row, col, obs.mark)
            score = get_heuristic_1(temp_grid, obs.mark, config)
            if score > best_score:
                best_score = score
                best_move = col

    return best_move
